# 서비스 진행상황 가시화
## 데이터준비

In [2]:
from NewsAnalyzer.PreProcesser import preprocesser

pp = preprocesser()

file_path = "data/2022-04-18 news data.csv"

df = pp.road_data(file_path)
df

,카테고리,언론사,기사제목,날짜,본문,기사링크
0,정치,SBS,[단독] '6억 집을 1억에'…이창용 한은 총재 후보자 다운계약 의혹,2022.04.18.,"\n\n\n\n\n이창용 한국은행 총재 후보자가 지난 2002년, 서울 중구 신당동...",https://news.naver.com//main/read.naver?mode=L...
1,정치,중앙일보,"노정희 선관위원장, '소쿠리 대선' 논란 44일만에 사퇴",2022.04.18.,\n\n\n\n\n\t\n\t 노정희 중앙선거관리위원회위원장이 18일 사의를 표명...,https://news.naver.com//main/read.naver?mode=L...
2,정치,YTN,단거리 발사에 김정은 참석...전술핵 탑재 노렸나,2022.04.18.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]김정은 국무위원장이 단...,https://news.naver.com//main/read.naver?mode=L...
3,정치,서울경제,"장제원 ""조작? 위조?…정호영, 조국과 뭐가 같나 얘기해보라""",2022.04.18.,\n\n\n\n\n장제원 대통령 당선인 비서실장이 지난 17일 서울 종로구 통의동 ...,https://news.naver.com//main/read.naver?mode=L...
4,정치,연합뉴스,"국민의힘·국민의당, 합당 공식 선언…""공동정부 초석""(종합2보)",2022.04.18.,"\n\n\n\n\n6·1 지방선거 단일 공천…당대표 이준석, 최고위원 2명 국민의당...",https://news.naver.com//main/read.naver?mode=L...
...,...,...,...,...,...,...
4641,세계,국제신문,전 세계 물가 폭등 아우성…30년만에 ‘大인플레’ 시대 진입,2022.04.18.,\n\n\n\n\nOECD 2월 상승률 7.7% 급등- 우크라發 에너지가격 큰 폭 ...,https://news.naver.com//main/read.naver?mode=L...
4642,세계,국민일보,물·전기 없이 13만명 ‘절망의 항전’… 마리우폴 ‘참극’,2022.04.18.,\n\n\n\n\n초토화 작전에 건물 90% 이상 파괴우크라군 최악 포위전 속 결사...,https://news.naver.com//main/read.naver?mode=L...
4643,세계,중앙일보,러 “마리우폴 함락” 주장…우크라 “방어군 버티고 있다”,2022.04.18.,\n\n\n\n\n 우크라이나 동남부 돈바스 지역에서 친러 분리주의 무장대원들이 ...,https://news.naver.com//main/read.naver?mode=L...
4644,세계,중앙일보,"침몰한 러의 자존심…미 “푸틴, 핵 쓸 가능성 커져”",2022.04.18.,\n\n\n\n\n\t\n\t 지난 14일 러시아 해군 흑해함대의 기함인 모스크바...,https://news.naver.com//main/read.naver?mode=L...


In [4]:
title = df["기사제목"]
news = df["본문"]
link = df["기사링크"]

In [6]:
link[0]

'https://news.naver.com//main/read.naver?mode=LSD&mid=shm&sid1=100&oid=055&aid=0000967628'

## 키워드 뽑기

In [2]:
from krwordrank.word import summarize_with_keywords

def listToString(str_list): #리스트를 문자열로 반환 (특수문자 제거 후 추출된 명사 입력)
    result = ""
    for s in str_list:
        result += s + " "
    string = result.strip()
    return string

def kwordrank(string): #해당 함수에서 불용어 제거 
    stop_words_list=[]
    stop_words_file = open("korean_stop_words.txt","r",encoding='utf-8')
    for line in stop_words_file.readlines():
        stop_words_list.append(line.rstrip())
        stop_words_file.close()
    stopwords=stop_words_list
    texts = [string]
    keywords = summarize_with_keywords(texts, min_count=10, max_length=10,beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
    passwords = {word:score for word, score in sorted(
        keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}
    return passwords


1. 전처리하지 않은 생 데이터 집어넣기

In [5]:
title_s = listToString(title)
news_s = listToString(news)
all_s = title_s + news_s

In [6]:
key_title = kwordrank(title_s)
key_title

scan vocabs ... 
num vocabs = 1101
done = 10


{'정호영': 9.957226149871792,
 '거리두기': 6.22287184028235,
 '김오수': 5.606336440965398,
 '해제': 4.26174710184484,
 '논란': 4.064001020538991,
 '검찰': 3.008096369660373,
 '의혹': 3.0008824856980145,
 '확진': 2.9155775822767973,
 '민주': 2.8484011884801745,
 '만에': 2.8404331050787817,
 '전면': 2.560069564299535,
 '후보': 2.4085800037461773,
 '이은해': 2.2507176281369454,
 '반발': 2.2389441291789605,
 '코로나': 2.0852023875026804,
 '내일부터': 2.0256288009854257,
 '사퇴': 1.892716220447157,
 '이유': 1.888664537552807,
 '서울': 1.8658056287432627,
 '10': 1.8493120513597612,
 '다시': 1.7878435245665534,
 'vs': 1.7598658002463154,
 '단일화': 1.7301535086369486,
 '청문회': 1.723452222189655,
 '부당행위': 1.7007477292535724,
 '일상': 1.698122944282494,
 '신규': 1.6979502998178062,
 '[다시': 1.6768526821385223,
 '편입': 1.6447008035575506,
 '의대': 1.6117163029079578,
 "'검수완박'": 1.5978693116894438,
 '경선': 1.5857027945633622,
 '금리': 1.5396977759022792,
 '최대': 1.5318544100552534,
 '"자녀': 1.5099548619622918,
 '한동훈': 1.5086542522500341,
 '송영길': 1.473996526500

In [7]:
key_news = kwordrank(news_s)
key_news

scan vocabs ... 
num vocabs = 38596
done = 10 Early stopped.


{'있다.': 100.50153811800209,
 '것으로': 65.2649502965833,
 '지난': 56.56742199831497,
 '있는': 45.405953412616405,
 '후보': 39.852905389165755,
 '17일': 33.75826674701608,
 '서울': 31.539460898068388,
 '대해': 30.376510515908297,
 '20': 30.089360959293828,
 '말했다.': 29.1979702094754,
 '통해': 27.858689167227727,
 '장관': 27.654130197649287,
 '것이': 27.633975424138573,
 '위해': 26.720407603323306,
 '검찰': 26.06125461492144,
 '따르면': 25.423173121399607,
 '했다.': 24.94398350370148,
 '대통령': 23.61079826845039,
 '이후': 23.565818977050963,
 '경우': 21.27557998810008,
 '밝혔다.': 21.246098796058543,
 '최근': 21.122888650727873,
 '이날': 20.271218342421676,
 '등을': 19.155184945039213,
 '관련': 19.093380601347622,
 '이어': 18.805656475987398,
 '국민': 18.513960132779886,
 '우크라이나': 17.12420443764913,
 '러시아': 17.11808275589738,
 '같은': 16.49040713184909,
 '미국': 16.422877905760924,
 '하는': 16.393959950624268,
 '다시': 15.730034854151882,
 '국회': 15.54579008026441,
 '당선인': 15.29013757060443,
 '그는': 14.949670727334926,
 '현재': 14.824493383132072,
 

In [8]:
key_all = kwordrank(all_s)
key_all

scan vocabs ... 
num vocabs = 39285
done = 10 Early stopped.


{'있다.': 100.56861079223785,
 '것으로': 65.23252747301646,
 '지난': 56.54094019281411,
 '있는': 45.56597389337606,
 '후보': 40.448020554122785,
 '17일': 33.977140255864576,
 '서울': 32.031058625811724,
 '대해': 30.38640995867097,
 '20': 30.3739921106222,
 '말했다.': 29.237960262804155,
 '통해': 27.898058188932986,
 '장관': 27.839947133858416,
 '것이': 27.70249958349568,
 '검찰': 26.937456145930366,
 '위해': 26.86204832844516,
 '따르면': 25.46833402226247,
 '했다.': 24.955310071611304,
 '대통령': 24.026478386831336,
 '이후': 23.70681090634634,
 '경우': 21.327175460461987,
 '밝혔다.': 21.22785016293019,
 '최근': 21.106717695830692,
 '이날': 20.269904322412255,
 '이어': 19.17036258135206,
 '등을': 19.13589452607535,
 '관련': 19.094188072270857,
 '국민': 18.922939975936536,
 '러시아': 17.41704761361893,
 '우크라이나': 17.182468467972836,
 '같은': 16.573233906539077,
 '미국': 16.495711351620333,
 '하는': 16.3547168253558,
 '다시': 16.081116112740933,
 '국회': 15.66707016754117,
 '당선인': 15.45722968278324,
 '그는': 14.956560313715825,
 '현재': 14.793078103950402,
 '10

2. 전처리 진행한 데이터 집어넣고 키워드 뽑기

In [9]:
title_n = pp.prep(file_path=file_path,title=True,tokenizing=False)
news_n = pp.prep(file_path=file_path,title=False,tokenizing=False)

training was done. used memory 0.770 Gb
all cohesion probabilities was computed. # words = 132468
all branching entropies was computed # words = 237707
all accessor variety was computed # words = 237707
training was done. used memory 0.771 Gb
all cohesion probabilities was computed. # words = 132468
all branching entropies was computed # words = 237707
all accessor variety was computed # words = 237707


In [10]:
title_s = listToString(title_n)
news_s = listToString(news_n)
all_s = title_s + news_s

In [11]:
key_title = kwordrank(title_s)
key_title

scan vocabs ... 
num vocabs = 1211
done = 10 Early stopped.


{'정호영': 5.689948449276507,
 '거리두기': 3.1907091025569456,
 '해제': 3.1677835039840803,
 '이은해': 2.9472664918731653,
 '김오수': 2.8213163844037883,
 '검찰': 2.533282951986473,
 '검수완박': 2.4992669110190286,
 '무기': 2.39303778158502,
 '시장': 2.367539316726547,
 '논란': 2.23129156012007,
 '종합': 2.1425252149450866,
 '민주': 2.0084541615123928,
 '서울': 1.9207971501709649,
 '의혹': 1.8622188354613267,
 '만에': 1.7545523729465873,
 '총장': 1.7344304053000288,
 '조현수': 1.7019826514698013,
 '자녀': 1.6910398773914368,
 '코로나': 1.6342797207607807,
 '금리': 1.6197816489567471,
 '10': 1.592579648239798,
 '한동훈': 1.5856718423113776,
 '확진': 1.555685136470891,
 '국민': 1.5476014902282147,
 '전면': 1.5340106310126473,
 '후보': 1.5088172977126715,
 '다시': 1.4631510118243503,
 '일상': 1.420470865662276,
 '아빠': 1.3534481117012107,
 '살인': 1.3318300493957524,
 '편입': 1.3193501452101928,
 '사퇴': 1.2984243661296002,
 '조사': 1.27272796866517,
 '이유': 1.2551838055997258,
 '마리우폴': 1.2333672202028763,
 '30': 1.2274500288747747,
 '계곡': 1.193169052635774,
 '

In [12]:
key_news = kwordrank(news_s)
key_news

scan vocabs ... 
num vocabs = 24784
done = 9 Early stopped.


{'20': 35.29871276711915,
 '하는': 34.29793804828815,
 '서울': 25.441161198094147,
 '지난': 23.640110074309746,
 '하고': 23.577881924423153,
 '후보': 19.813372527623134,
 '있는': 19.590831895143253,
 '한다': 19.499752046344888,
 '것으로': 18.54387802244817,
 '검찰': 16.124990614197948,
 '19': 14.947517754672797,
 '국민': 14.7278076045409,
 '것이': 13.460556593995419,
 '10': 13.395383264759364,
 '장관': 13.10315379979481,
 '하지': 12.874486697299806,
 '이라': 12.692715087519002,
 '대해': 12.64359032975895,
 '대통령': 11.863525148157304,
 '시장': 11.367374018997726,
 '위해': 11.24679433615131,
 '된다': 11.078585855229624,
 '코로나': 10.604798082904292,
 '한국': 10.272382297451678,
 '러시아': 10.018620174335286,
 '됐다': 9.968587044002296,
 '이어': 9.883742310414542,
 '되는': 9.871246269725322,
 '이후': 9.75203948603728,
 '7일': 9.74767181198736,
 '의원': 9.371423552806451,
 '따르면': 9.308318769217305,
 '관련': 9.12094535807744,
 '최근': 9.109471202133417,
 '통해': 9.090303069555597,
 '연합': 9.006564023222605,
 '국회': 8.7900985031352,
 '수사': 8.684257510959

In [13]:
key_all = kwordrank(all_s)
key_all

scan vocabs ... 
num vocabs = 25129
done = 9 Early stopped.


{'20': 35.3794135606469,
 '하는': 34.23518622365697,
 '서울': 25.66061714120222,
 '하고': 23.579922838434854,
 '지난': 23.558989440803053,
 '후보': 19.932781041215325,
 '한다': 19.529097978056903,
 '있는': 19.4859378740416,
 '것으로': 18.462727764984994,
 '검찰': 16.552248393009947,
 '19': 15.034068945812974,
 '국민': 14.940368835972984,
 '10': 13.764367166349416,
 '것이': 13.417314086976099,
 '장관': 13.128721342241711,
 '하지': 12.865711843202305,
 '이라': 12.648821213601114,
 '대해': 12.56495238386343,
 '대통령': 11.906903106608926,
 '시장': 11.789492292798995,
 '위해': 11.233545181318599,
 '된다': 11.02692052231228,
 '코로나': 10.929189422200702,
 '한국': 10.454963722154481,
 '러시아': 10.231087287083893,
 '됐다': 9.933062311152826,
 '이어': 9.923579225652977,
 '되는': 9.843399166686481,
 '7일': 9.748626482184031,
 '이후': 9.732134008980221,
 '따르면': 9.355984815666377,
 '의원': 9.345533712135971,
 '관련': 9.065591537672995,
 '최근': 9.060999244851482,
 '통해': 9.036210136492992,
 '연합': 8.975811175033176,
 '수사': 8.774716678120758,
 '국회': 8.7648367

-> 뉴스 본문이 들어가면 결과가 별로임
-> 기사 제목만을 가지고 키워드를 뽑는게 빠르고 성능이 좋음

In [15]:
keywords = key_title
keywords

{'정호영': 5.689948449276507,
 '거리두기': 3.1907091025569456,
 '해제': 3.1677835039840803,
 '이은해': 2.9472664918731653,
 '김오수': 2.8213163844037883,
 '검찰': 2.533282951986473,
 '검수완박': 2.4992669110190286,
 '무기': 2.39303778158502,
 '시장': 2.367539316726547,
 '논란': 2.23129156012007,
 '종합': 2.1425252149450866,
 '민주': 2.0084541615123928,
 '서울': 1.9207971501709649,
 '의혹': 1.8622188354613267,
 '만에': 1.7545523729465873,
 '총장': 1.7344304053000288,
 '조현수': 1.7019826514698013,
 '자녀': 1.6910398773914368,
 '코로나': 1.6342797207607807,
 '금리': 1.6197816489567471,
 '10': 1.592579648239798,
 '한동훈': 1.5856718423113776,
 '확진': 1.555685136470891,
 '국민': 1.5476014902282147,
 '전면': 1.5340106310126473,
 '후보': 1.5088172977126715,
 '다시': 1.4631510118243503,
 '일상': 1.420470865662276,
 '아빠': 1.3534481117012107,
 '살인': 1.3318300493957524,
 '편입': 1.3193501452101928,
 '사퇴': 1.2984243661296002,
 '조사': 1.27272796866517,
 '이유': 1.2551838055997258,
 '마리우폴': 1.2333672202028763,
 '30': 1.2274500288747747,
 '계곡': 1.193169052635774,
 '

### 임의의 키워드 하나를 선택하여 진행.

In [16]:
keyword = "거리두기"

키워드를 선정하면 비슷한 키워드를 뽑아내는 과정

In [43]:
import pandas as pd
from konlpy.tag import Okt
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def show_relevant_keyword(keyword, data): # 키워드와 사용할 뉴스 데이터를 인자로 입력받음
    
    # 에러를 막기 위해 'english'로 설정하였음. 
    # max_features는 5000개로 설정함
    tv = TfidfVectorizer(stop_words = 'english', max_features = 5000)
    x = tv.fit_transform(data)

    # words에는 feature가 된 단어들이 5000개 담겨 있음. 
    words = tv.get_feature_names() 

    # 불용어 리스트 불러오기
    stopwords = pd.read_csv('./korean_stop_words.txt', encoding = 'utf8')['아']

    ############## SVD 특이값 분해 ################
    from sklearn.decomposition import TruncatedSVD

    # n_components 는 max_features보다 적어야 함. 
    # 임의로 300개로 설정하였음
    svd = TruncatedSVD(n_components = 300, random_state = 1234)
    word_idx = words.index(keyword) # 사용자에게 입력받은 키워드의 인덱스 위치 확인
    svd.fit(x)

    kw_idx = svd.components_[:, word_idx].argmax()

    relevant_words_df = pd.DataFrame({'단어': words, 'loading': svd.components_[kw_idx]})
    
    ########### 상위 10가지 키워드 보여주기 ##########
    
    # 불용어 처리
    rel_words_df = relevant_words_df[relevant_words_df['단어'] != keyword]
    drop_index = []
    for i in rel_words_df['단어']:
        if i in list(stopwords):
            idx = rel_words_df[rel_words_df['단어'] == i].index[0]
            drop_index.append(idx)
    rel_words_df.drop(drop_index, inplace = True)
    
    # 상위 10개의 단어 추출
    rel_words_df = rel_words_df.sort_values('loading').tail(10)
    rel_words_df = rel_words_df.sort_values('loading', ascending = False)


    # 사용자가 입력한 키워드와 관련있는 상위 10개 단어를 데이터 프레임으로 반환
    return rel_words_df


In [50]:
# 전처리한 데이터 사용
rkw = show_relevant_keyword(keyword,title_n)

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [51]:
rkw

,단어,loading
4766,해제,0.453818
3463,전면,0.227297
4913,확진,0.212855
2545,신규,0.202448
1652,만에,0.201093
1105,내일부터,0.147106
14,10,0.139032
306,9만,0.125833
951,김오수,0.119848
645,검수완박,0.118290


In [20]:
show_relevant_keyword("정호영",df)

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,단어,loading
4976,후보자는,0.212344
3741,장관,0.187018
4974,후보자,0.175000
619,경북대,0.165813
3307,의대,0.162414
4975,후보자가,0.161736
2046,보건복지부,0.142043
3686,자녀,0.129352
4982,후보자의,0.119713
4619,편입,0.116163


In [21]:
title_n

['취임식 전날 靑 떠나 는 文 고민정 윤석열 잔인함 느껴 져',
 '조국 사태와 다르다 위법 없어 尹 정면 돌파 의지',
 '빨간 옷 입고 피켓 유승민 딸 국힘 출마 자 시험장에 뜬 까닭',
 '대구 수성 구청장 여론조사 김대 권 49 8 강민구 16 3 1대1 대결 구도',
 '조국 김건희 한동훈 살아있는 권력 수사 해야 내 딸은 중 학생 일기장 까지 압수',
 '정호영 아빠 찬스 해명 민주 입증 책임 국민 언론 에 돌려 비판',
 '편입 교육부 조사 아들 병역 진단 받겠다',
 '김오수 사표 내자 文에 임은정 검찰 총장 임명 요구 국민 청원',
 '국힘 공천시험장서 응원 피켓 든 여성 유승민 딸이 었다',
 '기자 정호영 신경전 사진제출않나 vs 기자 는 6년전 이름표 보관 하나',
 '집무실 이전 말고 뇌리에 남은 게 없다 인수위 존재 감 실종',
 '윤 40 년 지기 방어 내로남불 에 빠지 나',
 '정호영 정면 돌파 에 국민의힘 난감 이준석 엄밀히 평가 해야',
 '민주당 정호영 의혹 해소 안돼 한동훈 보이 콧 가능성',
 '스트 레이 트 한덕수 김앤 장에서 19 억 받고 무슨 일 했나',
 '정호영 부모 찬스 논란 에 곤혹스러 운 尹 팩트 확실히 있어야',
 '스트 레이 트 조국 판박이 비판 에도‥ 정호영 의 버티 기',
 '스트 레이 트 법무 장관 에 복심 한동훈 ‥ 윤석열 의 계산은',
 '김재원 한밤중 유영하 에 단일화 제안 엄중한 상황',
 '고민정 尹 취임식 전날 文 서울 모처 숙박 두고 잔인함 느껴 져',
 '민주 정호영 해명에도 아빠찬스 검증 고삐 의혹 해소 안돼',
 '국힘 국당 이르면 내일 합당 발표 선언 문 막바지 조율중 종합',
 '정치 톡톡 4 500 명 응시 안철수 내일 합당 정책실장 폐지',
 '문재인 정권 검찰 총장 잔혹 사 vs 정치적 반발 에 불과',
 '정호영 어떤 부 정도 없어 尹 팩트 확실 해야 직진 민주 소탐대실',
 '단식 김진태 경선 요구 유영하 단일화 수용',
 '정호영 자녀 의혹에 둔감 윤석열 의 구부러

In [25]:
show_relevant_keyword("거리두기",title_n)

C:\Users\tmdgh\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,단어,loading
4766,해제,0.453818
3463,전면,0.227297
4913,확진,0.212855
2545,신규,0.202448
1652,만에,0.201093
1105,내일부터,0.147106
14,10,0.139032
306,9만,0.125833
951,김오수,0.119848
645,검수완박,0.118290


현재까지 상황
1. 키워드 뽑음 -> 워드클라우드 그릴 수 있음
2. 키워드에서 관련키워드 뽑음 -> 얘도 워드클라우드 그릴수 있음
3. 본문요약은 BART를 사용하는데 본문 하나 넣으면 요약이 아주 잘됩니다.

문제
1. 키워드와 연관된 본문 뽑아오기 -> 기사제목에서 키워드 들어간거 가져오기. (진짜 단순하게 한다면)
2. 본문에서 단어 추출하고, 키워드 연관단어와 유사도 측정해서 기사 선정 -> 미친짓 why? 시간이 너무 오래걸림
3. 전체 기사를 전부 요약하는것도 시간이 너무 오래걸림.



In [38]:
from keybert import KeyBERT

kw = KeyBERT()

In [42]:
keywords = kw.extract_keywords(news_n[1], keyphrase_ngram_range=(1, 1), stop_words=None)
keywords

[('공정과', 0.4153),
 ('더불어민주당', 0.3997),
 ('전전긍긍', 0.3929),
 ('부정의', 0.386),
 ('당신의', 0.3584)]